In [ ]:
import numpy as np
import pandas as pd
import logging
import ipywidgets as widgets
import matplotlib.pyplot as plt
import itertools

In [ ]:
from sompy.sompy import SOMFactory

In [ ]:
import Parse_SOM
from tfprop_sompy.utils.data import calculate_SOM_radius

In [ ]:
logging.getLogger().setLevel(logging.WARNING)

In [ ]:
fin = 'data/Updated_PCM.csv'

In [ ]:
mapsize = (50, 50)
n_job = 4
mats_data_df = Parse_SOM.properties_dataframe_from_file(fin)

In [ ]:
# Replace spaces in the column names with underscores so they play nice with pandas
mats_data_df.columns = (*map(lambda x: x.replace(' ', '_'), mats_data_df.columns), )

In [ ]:
# This is just for reference, these are all the non-computed properties.
original_columns = ['Youngs_modulus',
 'Yield_strength',
 'Tensile_strength',
 'Compressive_strength',
 'Flexural_modulus',
 'Flexural_strength',
 'Shear_modulus',
 'Bulk_modulus',
 'Poisson_ratio',
 'Fatigue_strength',
 'Fracture_toughness',
 'Melting_point',
 'Max_service_temp',
 'Min_service_temp',
 'Thermal_conductivity',
 'Specific_heat_capacity',
 'Thermal_expansion_coefficient',
 'Latent_heat_of_fusion',
 'Electrical_resistivity',
 'Electrical_conductivity',
 'Galvanic_potential',
 'Mechanical_loss_coefficient']

In [ ]:
# Remove columns from here once you've either found them to be useless
# Or a definite "In" - put the ones you're keeping in "fixed_columns"
varying_columns = [
 'Electrical_conductivity',]

In [ ]:
# Place columns you've made a definite decision on including here
fixed_columns = [
 'Youngs_modulus',
 'Yield_strength',
 'Poisson_ratio',
 'Fatigue_strength',
 'Fracture_toughness',
 'Melting_point',
 'Thermal_conductivity',
 'Specific_heat_capacity',
 'Thermal_expansion_coefficient',
 'Latent_heat_of_fusion',]

In [ ]:
# Drop materials that don't have these properties
mats_data_df.dropna(subset=varying_columns+fixed_columns, inplace=True)

In [ ]:
# Fill in good PCM names here
good_pcms_list = ['Cadmium, commercial purity',
 'Copper-cadmium alloy, C16200, hard (h.c. copper)',
 'Copper-cadmium alloy, C16200, soft (h.c. copper)']

In [ ]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(len(s)+1))

In [ ]:
# Add the fixed columns to every iterated set
column_set_iterator = map(lambda x: fixed_columns + list(iter(x)), powerset(varying_columns))

In [ ]:
radius_df = pd.DataFrame(columns=['SOM_radius'])

for column_set in column_set_iterator:
    descr = mats_data_df[column_set].values
    sm = SOMFactory.build(descr, mapsize=mapsize, normalization='var', 
                          initialization='pca', component_names=column_set)
    # This will take some time
    sm.train(n_job=n_job, train_rough_len=0, train_finetune_len=0)
    
    radius_df.loc[", ".join(column_set)] = calculate_SOM_radius(mats_data_df, good_pcms_list, sm)[0]
    
    # Prevent too many maps from eating too much memory
    del sm
    

In [ ]:
radius_df